In [55]:
import keras
from keras.models import Model
from keras.layers import Input,Conv2D,MaxPooling2D,Add,Merge,UpSampling2D,Concatenate
from keras.utils import plot_model
from keras import backend as K

In [51]:
def get_model(weights_path=None):
    inputs = Input((224,224,3),name='input')
    conv3_64_1 = Conv2D(64,(3,3),activation = 'relu',padding='same',name='conv3_64_1')(inputs)
    conv3_64_2 = Conv2D(64,(3,3),activation='relu',padding='same',name='conv3_64_2')(conv3_64_1)
    pool2_1 = MaxPooling2D(pool_size=(2, 2),name='pool2_1')(conv3_64_2)
    
    conv3_128_1 = Conv2D(128,(3,3),activation='relu',padding='same',name='conv3_128_1')(pool2_1)
    conv3_128_2 = Conv2D(128,(3,3),activation='relu',padding='same',name='conv3_128_2')(conv3_128_1)
    pool2_2 = MaxPooling2D(pool_size=(2,2),name='pool2_2')(conv3_128_2)
    
    conv3_256_1 = Conv2D(256,(3,3),activation='relu',padding='same',name='conv3_256_1')(pool2_2)
    conv3_256_2 = Conv2D(256,(3,3),activation='relu',padding='same',name='conv3_256_2')(conv3_256_1)
    conv3_256_3 = Conv2D(256,(3,3),activation='relu',padding='same',name='conv3_256_3')(conv3_256_2)
    pool2_3 = MaxPooling2D(pool_size=(2,2),name='pool2_3')(conv3_256_3)
    
    conv3_512_1 = Conv2D(512,(3,3),activation='relu',padding='same',name='conv3_512_1')(pool2_3)
    conv3_512_2 = Conv2D(512,(3,3),activation='relu',padding='same',name='conv3_512_2')(conv3_512_1)
    conv3_512_3 = Conv2D(512,(3,3),activation='relu',padding='same',name='conv3_512_3')(conv3_512_2)
    
    conv1_256_add = Conv2D(256,(1,1),activation='relu',padding='same',name='conv1_256_add')(UpSampling2D(size = (2,2))(conv3_512_3))
    add_1 = Add()([conv1_256_add,conv3_256_3])
    
    conv3_128_add = Conv2D(128,(3,3),activation='relu',padding='same',name='conv3_128_add')(UpSampling2D(size = (2,2))(add_1))
    add_2 = Add()([conv3_128_add,conv3_128_2])
    
    conv3_64_add = Conv2D(64,(3,3),activation='relu',padding='same',name='conv3_64_add')(UpSampling2D(size = (2,2))(add_2))
    
    conv3_128_add_up = UpSampling2D(size=(2,2))(conv3_128_add)
    
    conv1_256_add_up = UpSampling2D(size=(2,2))(conv1_256_add)
    conv1_256_add_up = UpSampling2D(size=(2,2))(conv1_256_add_up)
    
    concat_1 = Concatenate(axis=3)([conv3_64_add,conv3_64_2])
    concat_2 = Concatenate(axis=3)([concat_1,conv3_128_add_up])
    concat_3 = Concatenate(axis=3)([concat_2,conv1_256_add_up])
    #merged = Merge([conv1_256_add, conv3_128_add,conv3_64_add,conv3_64_2], mode='concat')
    
    conv3_256_final = Conv2D(256,(3,3),activation='relu',padding='same',name='conv3_256_final')(concat_3)
    conv3_64_final1 = Conv2D(64,(3,3),activation='relu',padding='same',name='conv3_64_final1')(conv3_256_final)
    conv3_64_final2 = Conv2D(64,(3,3),activation='relu',padding='same',name='conv3_64_final2')(conv3_64_final1)
    
    output1 = Conv2D(50,(1,1),activation='softmax',name='output1')(conv3_64_final2)
    output2 = Conv2D(50,(1,1),activation='softmax',name='output2')(conv3_64_final2)
    
    model = Model(input=inputs,outputs=[output1,output2])
    
    if weights_path:
        model.load_weights(weights_path)
        
    return model
    

In [52]:
model = get_model()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:45: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


In [53]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv3_64_1 (Conv2D)             (None, 224, 224, 64) 1792        input[0][0]                      
__________________________________________________________________________________________________
conv3_64_2 (Conv2D)             (None, 224, 224, 64) 36928       conv3_64_1[0][0]                 
__________________________________________________________________________________________________
pool2_1 (MaxPooling2D)          (None, 112, 112, 64) 0           conv3_64_2[0][0]                 
__________________________________________________________________________________________________
conv3_128_

In [48]:
plot_model(model, to_file='model.png')

In [ ]:
def custom_loss(y_true,y_pred):
    return K.categorical_crossentropy(y_true[0], y_pred[0])+K.categorical_crossentropy(y_true[1], y_pred[1])

In [ ]:
adam_optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [ ]:
model.compile(optimizer=adam_optimizer,loss=custom_loss,metrics=['accuracy'])

In [ ]:
input_images_array = np.load('input_images.npy')
u_channel_array = np.array('u_channel_targets.npy')
v_channel_array = np.array('v_channel_targets.npy')

In [ ]:
model.fit([input_images_array], [u_channel_array, v_channel_array],
          epochs=50, batch_size=4)